In [ ]:
import torch
import deepxde as dde
import numpy as np
from typing import cast, Any
import plotly.graph_objects as go
import plotly

%load_ext jupyter_black

In [ ]:
def mae(y_true: np.ndarray, y_predict: np.ndarray) -> float:
    return np.mean(np.abs(y_true - y_predict))


def plot_result(train_state: dde.model.TrainState, name_case: str) -> go.Figure:
    plotly_colors = plotly.colors.qualitative.Plotly
    width = 600
    height = 600
    scale = 3

    fig = go.Figure()
    test_idx_sorted = np.argsort(train_state.X_test[:, 0])
    fig.add_trace(
        go.Scatter(
            x=train_state.X_test[test_idx_sorted, 0],
            y=train_state.y_pred_test[test_idx_sorted, 0],
            mode="lines",
            name="Prediction",
            line=dict(color=plotly_colors[0], width=3),
        )
    )
    fig.add_trace(
        go.Scatter(
            x=train_state.X_test[test_idx_sorted, 0],
            y=train_state.y_test[test_idx_sorted, 0],
            mode="lines",
            name="Ground truth",
            line=dict(color=plotly_colors[1], dash="dash"),
        )
    )

    train_idx_sorted = np.argsort(train_state.X_train[:, 0])
    fig.add_trace(
        go.Scatter(
            x=train_state.X_train[train_idx_sorted, 0],
            y=train_state.y_train[train_idx_sorted, 0],
            mode="markers",
            name="Training points",
            marker=dict(color=plotly_colors[0], size=8),
        )
    )
    error = mae(train_state.y_test[:, 0], train_state.y_pred_test[:, 0])
    fig.update_layout(
        autosize=False,
        width=width,
        height=height,
        title=dict(
            text=f"{name_case.capitalize()} case, MAE error {error:.2E}",
            y=0.91,
            x=0.5,
            xanchor="center",
            yanchor="bottom",
        ),
        xaxis=dict(title=dict(text="t")),
        yaxis=dict(title=dict(text="y(t)")),
        legend=dict(
            orientation="h",
            # yanchor="top",
            # y=1,
            # xanchor="right",
            # x=1,
            yanchor="bottom",
            y=1,
            xanchor="right",
            x=1,
        ),
        font=dict(size=14),
        margin=dict(l=40, r=40, t=90, b=60),
    )
    fig.write_image(
        f"../figs/harmonic_oscillator_{name_case}_xi0.png",
        width=width,
        height=height,
        scale=scale,
    )
    return fig

In [ ]:
class HarmonicOscillator0D:
    """Class modeling the harmonic oscillator in 0+1 dimensions. The ODE is:
    $$
        \\frac{\\partial^2 y}{\\partial t^2} + \\xi_0 * \\omega_0 * \\frac{\\partial y}{\\partial t} + \\frac{\\omega_0^2}{4} y = 0
    $$
    and with initial conditions $y(t = 0) = y_0, y^\\prime(t = 0) = y_1$.


    Args:
        omega0 (float): Intrinsic frequency of the system.
        xi0 (float): Fluid friction term.
        y0 (float): Initial value of the solution.
        y1 (float): Initial derivative of the solution.
    """

    __slots__ = ("xi0", "omega0", "xi", "omega", "y0", "y1")

    def __init__(self, xi0: float, omega0: float, y0: float, y1: float):
        # ODE parameters
        self.xi0 = xi0
        self.omega0 = omega0
        self.xi = xi0 * omega0 / 2
        self.omega = self.omega0 * np.sqrt(np.abs(np.power(self.xi0, 2) - 1)) / 2

        # Initial value condition
        self.y0 = y0
        self.y1 = y1

    def equation(self, x: torch.Tensor, y: torch.Tensor) -> torch.Tensor:
        """Defines the ODE.

        Args:
            x (torch.Tensor): Input of the neural network.
            y (torch.Tensor): Output of the neural network.

        Returns:
            (torch.Tensor): Residual of the differential equation.

        """
        dy_dt = cast(torch.Tensor, dde.grad.jacobian(y, x, i=0, j=0))
        dy2_dt2 = cast(torch.Tensor, dde.grad.hessian(y, x, i=0, j=0))
        ode = dy2_dt2 + self.xi0 * self.omega0 * dy_dt + (self.omega0**2 / 4) * y
        return ode

    def initial_value(self, x: torch.Tensor) -> torch.Tensor:
        """Defines the initial value for the differential equation.

        Args:
            x (torch.Tensor): Input of the neural network.

        Returns:
            (torch.Tensor): Value at the boundary for the initial condition.
        """
        return y0

    def initial_value_derivative(
        self, x: torch.Tensor, y: torch.Tensor, X: np.ndarray
    ) -> torch.Tensor:
        """Defines the initial value of the derivative for the differential equation.

        Args:
            x (torch.Tensor): Input of the neural network.
            y (torch.Tensor): Output of the neural network.
            X (np.ndarray): Numpy array of the inputs.

        Returns:
            (torch.Tensor): Residual of the initial value condition for the derivative.
        """
        dy_dt = cast(torch.Tensor, dde.grad.jacobian(y, x, i=0, j=0))
        return dy_dt - self.y1

    def exact_solution(self, x: np.ndarray) -> np.ndarray:
        """Defines the ground truth solution for the system.

        Args:
            x (np.ndarray): Points on which to evaluate the solution.

        Returns:
            (np.ndarray): Value of the solution on the points.
        """

        # Damping term
        exp_term = np.exp(-self.xi * x)
        # Critical solution
        if self.xi0 == 1:
            c0 = self.y0
            c1 = self.y1 + self.xi * self.y0
            return exp_term * (c0 + c1 * x)
        else:
            c0 = self.y0
            c1 = (self.y1 + self.xi * self.y0) / self.omega
            # Underdamped case
            if self.xi0 < 1:
                return exp_term * (
                    c0 * np.cos(self.omega * x) + c1 * np.sin(self.omega * x)
                )
            # Overdamped case
            else:
                return exp_term * (
                    c0 * np.cosh(self.omega * x) + c1 * np.sinh(self.omega * x)
                )

In [ ]:
# Equation parameters

omega0 = 2 * np.pi * 5
y0 = 1
y1 = 0

# Neural network parameters
input_size = [1]
output_size = [1]
layers_sizes = input_size + [64, 128, 64] + output_size
activation = "tanh"
initializer = "Glorot normal"

# Training parameters
optimizer_kw = dict(
    lr=0.001,
    metrics=["l2 relative error"],
    loss_weights=[0.001, 1, 0.5],
)
n_iterations = 10_000

# Mesh parameters
n_training_inside = 64
n_training_bdy = 2
n_training_initial = 2
n_test = 500


def solve_problem(xi0: float, omega0: float, y0: float, y1: float) -> None:
    """Given the parameters, defines the model and trains the model."""
    # System
    ho = HarmonicOscillator0D(xi0=xi0, omega0=omega0, y0=y0, y1=y1)

    # Geometry description
    geom = dde.geometry.TimeDomain(0, 1)

    def boundary_t0(x: torch.Tensor, on_boundary: bool) -> bool:
        return on_boundary and dde.utils.isclose(x[0], 0)

    # Initial value conditions
    ic_value = dde.icbc.IC(geom, ho.initial_value, lambda _, on_initial: on_initial)
    ic_derivative_value = dde.icbc.OperatorBC(
        geom, ho.initial_value_derivative, boundary_t0
    )

    # Load data
    data = dde.data.TimePDE(
        geom,
        ho.equation,
        [ic_value, ic_derivative_value],
        num_domain=n_training_inside,
        num_boundary=n_training_bdy,
        # num_initial=n_training_initial,
        solution=ho.exact_solution,
        num_test=n_test,
    )

    neural_net = dde.nn.FNN(layers_sizes, activation, initializer)
    model = dde.Model(data, neural_net)
    model.compile("adam", **optimizer_kw)

    losshistory, train_state = model.train(iterations=n_iterations)
    return losshistory, train_state

In [ ]:
xi0 = 0.1
# Training parameters
optimizer_kw = dict(
    lr=0.001,
    metrics=["l2 relative error"],
    loss_weights=[0.001, 1, 0.5],
)
loss_history, train_state = solve_problem(xi0=xi0, omega0=omega0, y0=y0, y1=y1)
dde.saveplot(loss_history, train_state, issave=False, isplot=True)
plot_result(train_state, name_case="underdamped")

In [ ]:
xi0 = 1.5
# Training parameters
optimizer_kw = dict(
    lr=0.001,
    metrics=["l2 relative error"],
    loss_weights=[0.001, 1, 0.8],
)
loss_history, train_state = solve_problem(xi0=xi0, omega0=omega0, y0=y0, y1=y1)
dde.saveplot(loss_history, train_state, issave=False, isplot=True)
plot_result(train_state, name_case="overdamped")

In [ ]:
xi0 = 1.0
optimizer_kw = dict(
    lr=0.001,
    metrics=["l2 relative error"],
    loss_weights=[0.001, 1, 0.8],
)
loss_history, train_state = solve_problem(xi0=xi0, omega0=omega0, y0=y0, y1=y1)
dde.saveplot(loss_history, train_state, issave=False, isplot=True)
plot_result(train_state, name_case="critical")

In [ ]:
xi0 = 0.0
optimizer_kw = dict(
    lr=0.001,
    metrics=["l2 relative error"],
    loss_weights=[0.001, 1, 0.8],
)
loss_history, train_state = solve_problem(xi0=xi0, omega0=omega0, y0=y0, y1=y1)
dde.saveplot(loss_history, train_state, issave=False, isplot=True)
plot_result(train_state, name_case="frictionless")